In [ ]:
# default_exp config
#%load_ext autoreload
from nbdev.showdoc import show_doc
from utilities.ipynb_docgen import *


# Configuration data and basic functions
> Basic functions and configuration stuff

In [ ]:
#collapse-hide
def summary():
    """
    
    ##### config.Config -- parameters

    for now, but could be obtained from an editable yaml file.
    
    {config_text}
    

    These are dataclass classes, and can be initialized with
    parameters. For example, to suppress printout 
        
    ```
    config = Config(verbose=0)
    ```
    
    ##### config.cache -- a file cache
    The class `Cache`, available from `config.cache` implements a file cache in the folder
    `config.files.cachepath`
    
    {cache_text}
    """
    config = Config()
    config_text = monospace(config, summary='config parameter list')

 
    cache_text = monospace(config.cache, 'cache contents' )
    assert files.valid
    return locals()
#nbdoc(summary) 


In [ ]:
#export
from astropy.time import Time
from astropy.coordinates import SkyCoord
from dataclasses import dataclass
from pathlib import Path
from typing import Tuple
import os, sys
import numpy as np
import pickle

In [ ]:
#export
class Cache(dict):
    """
    Manage a file cache

    - `path` -- string or `filepath` object <br> This is the folder where the index and data files are saved.
    - `clear` -- set True to clear the cache on initialization

    This uses pickle to save objects, associated with a hashable key, which is used to index the
    filename in a file `index.pkl` in the same folder.

    The `__call__` function is a convenient way to use it, so one call may either store a new entry or retrieve an existing one.

    """

    def __init__(self, path, clear:bool=False):


        self.path = Path(path) if path else None
        if not self.path: return
        if not self.path.exists() :
            raise Exception(f'cache Path {self.path} does not exist ')

        self.index_file = self.path/'index.pkl'

        if self.path.exists():
            if clear:
                print('Clearing cache!')
                self.clear()
            else:
                self._load_index()

    def _dump_index(self):
        with open(self.index_file, 'wb') as file:
            pickle.dump(self, file)

    def _load_index(self):
        if not self.index_file.exists():
            self._dump_index()
            return
        with open(self.index_file, 'rb') as file:
            self.update(pickle.load(file))

    def add(self, key, object, exist_ok=False):
        if not self.path: return
        assert type(key)==str, f'Expect key to be a string, got {key}'
        if key  in self:
            if not exist_ok:
                print(f'Warning: cached object for key "{key}" exists', file=sys.stderr)
            filename = self[key]
        else:
            filename = self.path/f'cache_file_{hex(key.__hash__())[3:]}.pkl'
            self[key] = filename
            self._dump_index()

        with open(filename, 'wb') as file:
            pickle.dump(object, file )


    def get(self, key):
        if key not in self:
            return None
        filename = self[key]
        if not filename.exists():
            # perhaps deleted by another instance?
            print(f'File for Cache key {key} not found, removing entry', file='sys.stderr')
            selt.pop(key)
            return None
        with open(filename, 'rb') as file:
            ret = pickle.load(file)
        return ret

    def clear(self):
        if not self.path: return
        for f in self.path.iterdir():
            if f.is_file:
                f.unlink()
        super().clear()

        self._dump_index()

    def remove(self, key):
        """remove entry and associated file"""
        if not self.path: return
        if key not in self:
            print(f'Cache: key {key} not found', file=sys.stderr)
            return
        filename = self[key]
        try:
            filename.unlink()
        except:
            print(f'Failed to unlink file {filename}')
        super().pop(key)
        self._dump_index()


    def __call__(self, key, func, *pars, description='', overwrite=False, **kwargs,
                ):
        """
        One-line usage interface for cache use

        - `key` -- key to use, usually a string. Must be hashable <br>
            If None, ignore cache and return the function evaluation
        - `func` -- user function that will return an object that can be pickled
        - `pars`, `kwargs` -- pass to `func`
        - `description` -- optional string that will be printed
        - `overwrite` -- if set, overwrite previous entry if exists

        Example:
        <pre>
        mycache = Cache('/tmp/thecache', clear=True)

        def myfun(x):
            return x

        result = mycache('mykey', myfun, x=99,  description='My data')

        </pre>

        """

        if key is None:
            return func(*pars, **kwargs)

        ret = self.get(key)
        if description:
            print(f'{description}: {"Saving to" if key not in self or overwrite else "Restoring from"} cache', end='')
            print('' if key == description else f' with key "{key}"')

        if ret is None or overwrite:
            ret = func(*pars, **kwargs)
            self.add(key, ret, exist_ok=overwrite)
        return ret

    def show(self, starts_with=''):
        import datetime
        if not self.path: return 'Cache not enabled'
        title = 'Cache contents' if not starts_with else f'Cache entries starting with {starts_with}'
        s = f'{title}\n {"key":30}   {"size":>10}  {"time":20} {"name"}, in folder {self.path}\n'
        for name, value in self.items():
            if name is None or not name.startswith(starts_with) : continue
            try:
                stat = value.stat()
                size = stat.st_size
                mtime= str(datetime.datetime.fromtimestamp(stat.st_mtime))[:16]
                s += f'  {name:30s}  {size:10}  {mtime:20} {value.name}\n'
            except Exception as msg:
                s += f'{name} -- file not found'
        return s

    def __str__(self):
        return self.show()

In [ ]:
p = Path('/tmp/cache')
p.mkdir?

Signature: p.mkdir(mode=511, parents=False, exist_ok=False)
Docstring: Create a new directory at this given path.
File:      ~/miniconda3/lib/python3.7/pathlib.py
Type:      method


In [ ]:
show_doc(Cache.__call__)

<h4 id="Cache.__call__" class="doc_header"><code>Cache.__call__</code><a href="__main__.py#L96" class="source_link" style="float:right">[source]</a></h4>

> <code>Cache.__call__</code>(**`key`**, **`func`**, **\*`pars`**, **`description`**=*`''`*, **`overwrite`**=*`False`*, **\*\*`kwargs`**)

One-line usage interface for cache use

- `key` -- key to use, usually a string. Must be hashable <br>
    If None, ignore cache and return the function evaluation
- `func` -- user function that will return an object that can be pickled
- `pars`, `kwargs` -- pass to `func`
- `description` -- optional string that will be printed
- `overwrite` -- if set, overwrite previous entry if exists

Example:
<pre>
mycache = Cache('/tmp/thecache', clear=True)

def myfun(x):
    return x

result = mycache('mykey', myfun, x=99,  description='My data')

</pre>

### Cache test

In [ ]:
#collapse_hide
def cache_test(path):
    c = Cache(path, clear=True)

    # simmple interface
    c.add('one', 'one');
    c.add('two', 'two')
    c.add('two', 'two') # getnerates warning
    if path is not None:
        assert c.get('two') == 'two'

    # test function interface
    func = lambda x:f'value: {x}'
    
    r1 = c('four',  func,  4, description='Test')
    r2 = c('four',  func,  5,  description='Test') #should not get called
    assert c.path is None or r1==r2, f'{r1}, {r2}'
    
    # remaving an entry
    print(f'Before remove:\n{c}')
    assert 'four' in c
    c.remove('four')
    assert 'four' not in c
   
    
    c.clear()

test_path = Path('/tmp/cache_test').mkdir(exist_ok=True)
#??cache_test(test_path)
# disabled should ignore
#cache_test(None)

In [ ]:
# export
@dataclass
class Config:
    """Default light curve configuration parameters"""
    verbose : int = 1

    # data source: if set, expect all data files here
    wtlike_data:  str = '~/wtlike_data' # wired in for convenience

    # cache
    cachepath: str = '~/wtlike_cache'

    radius: float = 4
    cos_theta_max:float=0.4
    z_max : float=100

    # binning
    energy_edges = np.logspace(2,6,17)
    etypes = (0,1)

    week_range: Tuple=(None,None)

    time_bins: Tuple=(0,0,7)

    use_uint8: bool=False  # for weights

    # healpix data representation used by data
    nside : int=1024
    nest: bool=True

    # exposure calculation
    bins_per_decade: int=5
    base_spectrum: str='lambda E: (E/1000)**-2.1'
    energy_range: Tuple = (100.,1e6)

    # analysis
    likelihood_rep: str='poisson'

    def __post_init__(self):
        # set up data
        self.errors=[]
        if self.wtlike_data is None:
            self.errors.append('wtlike_data must be set')
        if self.cachepath is None:
            self.errors.appnd('wtlike_cache must be set')
        self.wtlike_data = df = Path(self.wtlike_data).expanduser()
        self.cachepath =  Path(self.cachepath).expanduser()
        if not self.wtlike_data.is_dir() and not self.wtlike_data.is_symlink():
            self.errors.append(f'data_folder {df} not a directory or symlink')
        subs = 'aeff_files weight_files data_files'.split()
        for sub in subs:
            if not (df/sub).is_dir() and (df/sub).is_symlink() :
                self.errors.append(f'{df/sub} is not a directory or symlink')
        if self.verbose>1:
            print(self)

    @property
    def cache(self):
        if not hasattr(self, '_cache'):
            self._cache = Cache(self.cachepath, clear=False)
        return self._cache

    @property
    def valid(self):
        return len(self.errors)==0

    def __str__(self):
        s = 'Configuration parameters \n'
        for name, value in self.__dict__.items():
            if name=='files' or name.startswith('_'): continue
            s += f'  {name:15s} : {value}\n'
        return s

    def __repr__(self): return str(self)

## Time conversion

- MET: mission elapsed time
- MJD: modified Julian date (days)
- UTC: ISO time

In [ ]:
#export

day = 24*3600.
first_data=54683

def MJD(met):
    "convert MET to MJD"
    #mission_start = Time('2001-01-01T00:00:00', scale='utc').mjd
    # From a FT2 file header
    # MJDREFI =               51910. / Integer part of MJD corresponding to SC clock S
    # MJDREFF =  0.00074287037037037 / Fractional part of MJD corresponding to SC cloc
    mission_start = 51910.00074287037
    return (mission_start + met/day  )

def UTC(mjd):
    " convert MJD value to ISO date string"
    t=Time(mjd, format='mjd')
    t.format='iso'; t.out_subfmt='date_hm'
    return t.value

In [ ]:
assert UTC(MJD(0))=='2001-01-01 00:01'

### Miscellaneous utilities

In [ ]:
#export
def bin_size_name(bins):
    """Provide a nice name, e.g., 'day' for a time interval
    """
    if np.isscalar(bins) :
        binsize = bins
    else:
        binsize = np.mean(bins)

    def check_unit(x):
        unit_table = dict(week=1/7, day=1, hour=24, min=24*60, s=24*3600)
        for name, unit in unit_table.items():
            t = x*unit
            r = np.mod(t+1e-9,1)
            if r<1e-6 or t>1:
                return t, name
        return x, 'day'
    n, unit =  check_unit(binsize)
    nt = f'{n:.0f}' if np.mod(n,1)<1e-2 else f'{n:.0f}'
    return f'{nt}-{unit}'# if n>1 else f'{unit}'

In [ ]:

# print([bin_size_name(x) for x in [0.25, 1, 2, 1/24, 2/24, 2.1/24/60, 14]])
# bin_size_name(np.linspace(0,10,11))

In [ ]:
#export                
class PointSource():
    """Manage the position and name of a point source
    """
    def __init__(self, name, position=None):
        """position: (l,b) tuple or None. if None, expect to be found by lookup
        """
        self.name=name
        if position is None:
            try:
                skycoord = SkyCoord.from_name(name)
            except Exception as e:
                print(f'PointSource: a source "{name}" was not recognized by SkyCoord: {e}', file=sys.stderr)
                raise
            gal = skycoord.galactic
            self.l,self.b = (gal.l.value, gal.b.value)
        else:
            self.l,self.b = position
            skycoord = SkyCoord(self.l,self.b, unit='deg', frame='galactic')
        self.skycoord = skycoord
    def __str__(self):
        return f'Source "{self.name}" at: (l,b)=({self.l:.3f},{self.b:.3f})'
    def __repr__(self): return str(self)

    @property
    def ra(self):
        sk = self.skycoord.transform_to('fk5')
        return sk.ra.value
    @property
    def dec(self):
        sk = self.skycoord.transform_to('fk5')
        return sk.dec.value

    @property
    def filename(self):
        """Modified name for file system"""
        return self.name.replace(' ', '_').replace('+','p')

    @classmethod
    def fk5(cls, name, position):
        """position: (ra,dec) tuple """
        ra,dec = position
        sk = SkyCoord(ra, dec, unit='deg',  frame='fk5').transform_to('galactic')
        return cls(name, (sk.l.value, sk.b.value))

In [ ]:
show_doc(PointSource.fk5)

<h4 id="PointSource.fk5" class="doc_header"><code>PointSource.fk5</code><a href="__main__.py#L39" class="source_link" style="float:right">[source]</a></h4>

> <code>PointSource.fk5</code>(**`name`**, **`position`**)

position: (ra,dec) tuple 

In [ ]:
for s, expect in [( PointSource('Geminga'),             'Source "Geminga" at: (l,b)=(195.134,4.266)'),
                  ( PointSource('gal_source', (0,0)),   'Source "gal_source" at: (l,b)=(0.000,0.000)', ),
                  ( PointSource.fk5('fk5_source',(0,0)),'Source "fk5_source" at: (l,b)=(96.337,-60.189)',)
                   ]:    
    assert str(s)==expect, f'expected {expect}, got {str(s)}'


In [ ]:
# export
def decorate_with(other_func):
    def decorator(func):
        func.__doc__ += other_func.__doc__
        return func
    return decorator

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()
!date

Converted 00_config.ipynb.
Converted 01_data_man.ipynb.
Converted 02_effective_area.ipynb.
Converted 03_weights.ipynb.
Converted 04_exposure.ipynb.
Converted 04_simulation.ipynb.
Converted 05_source_data.ipynb.
Converted 06_poisson.ipynb.
Converted 07_loglike.ipynb.
Converted 08_cell_data.ipynb.
Converted 09_lightcurve.ipynb.
Converted 14_bayesian.ipynb.
Converted 90_main.ipynb.
Converted 99_tutorial.ipynb.
Converted index.ipynb.
Thu May 20 06:28:04 PDT 2021


### Cache is a mixin for a class ?

In [ ]:
# from pathlib import Path
# import pickle
# config = Config(cachepath='/tmp/cache')
# config.cache

# !ls -l /tmp/cache
# ! rm -f /tmp/cache/index.pkl

In [ ]:
# class CacheMixin(object):
#     def __init__(self, cache):
#         self.cache = cache

# class TestCache(CacheMixin):
#     def __init__(self,  config):
#         Path('/tmp/cache').mkdir(exist_ok=True)
#         CacheMixin.__init__(config.cache)
       
# tc = TestCache(config)

TypeError: __init__() missing 1 required positional argument: 'cache'